In [1]:
import numpy as np
import pandas as pd
from flask import Flask, request, jsonify, render_template
import pickle
from sklearn.preprocessing import MinMaxScaler

In [2]:
app = Flask(__name__)

In [3]:
model = pickle.load(open('model.pkl', 'rb'))

In [4]:

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict',methods=['POST'])
def predict():

    parameters = ['sysBP','glucose','age','cigsPerDay','totChol','diaBP','prevalentHyp','male','BPMeds','diabetes','TenYearCHD']
    feature = [float(x) for x in request.form.values()]
    features = np.array(feature).reshape(1,-1)
    
    scaler = pickle.load(open('scaler_model.pkl', 'rb'))
    
    scaled_features = scaler.transform(features) 
    feature_df = pd.DataFrame(dict(zip(parameters, scaled_features.ravel())), index = [0])
    prediction = model.predict(feature_df)
    
    
    print("feature", feature)
    print("final features",scaled_features)
    print("prediction:",prediction)
    output = round(prediction[0], 2)
    print(output)

    if output == 0:
        return render_template('index.html', prediction_text='THE PATIENT IS NOT LIKELY TO HAVE A HEART DISEASE')
    else:
         return render_template('index.html', prediction_text='THE PATIENT IS LIKELY TO HAVE A HEART DISEASE')
        
@app.route('/predict_api',methods=['POST'])
def results():

    data = request.get_json(force=True)
    prediction = model.predict([np.array(list(data.values()))])

    output = prediction[0]
    return jsonify(output)

In [5]:
if __name__ == "__main__":
    app.run(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Aug/2021 09:57:21] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Aug/2021 09:57:21] "GET /static/style.css HTTP/1.1" 404 -
127.0.0.1 - - [10/Aug/2021 09:57:21] "GET /static/style.css HTTP/1.1" 404 -
127.0.0.1 - - [10/Aug/2021 09:57:29] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [10/Aug/2021 09:57:29] "GET /static/style.css HTTP/1.1" 404 -


feature [10.0, 200.0, 80.0, 0.0, 120.0, 70.0, 0.0, 0.0, 0.0, 0.0]
final features [[-0.44680851  0.4519774   1.30555556  0.         -0.01176471  0.23280423
   0.          0.          0.          0.        ]]
prediction: [0]
0


127.0.0.1 - - [10/Aug/2021 09:58:36] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [10/Aug/2021 09:58:36] "GET /static/style.css HTTP/1.1" 404 -


feature [185.0, 110.0, 62.0, 0.0, 276.0, 95.0, 1.0, 0.0, 0.0, 0.0]
final features [[0.61702128 0.19774011 0.80555556 0.         0.44705882 0.4973545
  1.         0.         0.         0.        ]]
prediction: [1]
1
